# Calculation of area-integrated time series of EKE in selected ocean basins
EKE in different ocean basins may have varying responses to climate change with varying physical drivers. To assess each ocean basin individually, area-integrated time series of EKE in selected ocean basins will be calculate. The ocean basins correspond to the major eddy-rich regions of the global ocean, and tend to follow major ocean currents.

In [ ]:
import xarray as xr
import pyfesom2 as pf
import numpy as np
from joblib import Parallel, delayed

In [ ]:
meshpath='/path/to/mesh/data/'

In [ ]:
#load mesh info
mesh=pf.load_mesh(meshpath)
griddes=xr.open_dataset(meshpath+'griddes.nc')

Identify the indices corresponding to cells within each basin.

In [ ]:
#load the reduced basins
braz_basin=np.load('braz_basin_index.npy')
kuro_basin=np.load('kuro_basin_index.npy')
gulf_basin=np.load('gulf_basin_index.npy')
east_basin=np.load('east_basin_index.npy')
agul_basin=np.load('agul_basin_index.npy')
accu_basin=np.load('accu_basin_index.npy')

In [ ]:
#define some lists
basins=[gulf_basin,kuro_basin,east_basin,agul_basin,braz_basin,accu_basin]
names=['gulf_basin_ts','kuro_basin_ts','east_basin_ts','agul_basin_ts','braz_basin_ts','accu_basin_ts']
runs=['r1','r2','r3','r4','r5']

During area weighting, EKE (m^2/s^2) is multiplied by cell area (m^2). Time series then take the sum of the basins. 

To return units to m^2/s^2 we could divide time series values sum by the total area of each basin. This would allow better comparison between basins but the spatial density of EKE in each basin would still confound the comparison (i.e. low-EKE areas in basin corners). To avoid problems with spatial density, we could use only the sums of EKE, but this would be confounded by total basin area. Our time series will be standardized making this choice unnecessary; therefore, the produce_ts function will only calculate the sum for efficiency.

In [ ]:
def produce_ts(k):
    return temp.w_EKE[k,basin].sum(dim='ncells').values

In [ ]:
#produce modeled time series
for i in np.arange(len(basins)):
    box=boxes[i]
    for j in np.arange(len(runs)):
        temp=xr.open_dataset('weighted/c6_ssh-eke_mon_lin_eq_5day_node_'+runs[j]+'_1860-2090_nol_21yr_anom_runmean_rename_weighted.nc')
        ts=Parallel(n_jobs=-1,batch_size=200,verbose=10)(delayed(produce_ts)(k) for k in np.arange(len(temp.time)))
        ts=np.asarray(ts)
        np.save(names[i]+'_'+runs[j]+'.npy',ts)

In [ ]:
#produce observed time series
for i in np.arange(len(basins)):
    box=boxes[i]
    temp=xr.open_dataset('obs_eke_1993-2020_nol_5d_rm_runmean_mdata_weighted.nc')
    ts=Parallel(n_jobs=-1,batch_size=20,verbose=10)(delayed(produce_ts)(k) for k in np.arange(len(temp.time)))
    ts=np.asarray(ts)
    np.save(names[i]+'_obs.npy',ts)

In [ ]:
i=0
basin=basins[i]
temp=xr.open_dataset('obs_eke_1993-2020_nol_5d_rm_runmean_mdata_weighted.nc')
ts=Parallel(n_jobs=-1,batch_size=20,verbose=10)(delayed(produce_ts)(k) for k in np.arange(len(temp.time)))
ts=np.asarray(ts)
np.save(names[i]+'_obs.npy',ts)